## Download from RShiny

In [1]:
"""
Download LA County neighborhood testing data.
Download persons tested and tests performed.
Download from RShiny.
No date column, it probably reflects 2 days ago, given their comment in sidebar (community testing).
"""
import pandas as pd
import pytz

from datetime import datetime, timedelta

NEIGHBORHOOD_TESTING_URL = (
    "https://lacdph.shinyapps.io/covid19_surveillance_dashboard/_w_6f8c1821/"
    "session/e7aba2d098b696a635925820c86d0dbc/download/download4?w=6f8c1821"
)
  
TESTS_PERFORMED = (
    "https://lacdph.shinyapps.io/covid19_surveillance_dashboard/_w_6f8c1821/"
    "session/e7aba2d098b696a635925820c86d0dbc/download/download6?w=6f8c1821"
)

PERSONS_TESTED = (
    "https://lacdph.shinyapps.io/covid19_surveillance_dashboard/_w_6f8c1821/"
    "session/e7aba2d098b696a635925820c86d0dbc/download/download5?w=6f8c1821"   
)

S3_FILE_PATH = "s3://public-health-dashboard/jhu_covid19/"

# Neighborhood Testing data
def download_from_rshiny():
    df = pd.read_csv(NEIGHBORHOOD_TESTING_URL)

    df = (df.assign(
            download_date = (datetime.today()
                    .astimezone(pytz.timezone("US/Pacific"))
                    .date()
            ),
            date = (datetime.today()
                    .astimezone(pytz.timezone("US/Pacific"))
                    .date() - timedelta(days=2)
            )
        ).rename(columns = {"geo_merge": "neighborhood"})
    )

    df = (df.assign(
            download_date = pd.to_datetime(df.download_date),
            date = pd.to_datetime(df.date)
        ).drop(columns = "Unnamed: 0")
    )

    file_name = datetime.today().astimezone(pytz.timezone("US/Pacific")).date()

    df.to_parquet(f"{S3_FILE_PATH}la-county-neighborhood-testing-{file_name}.parquet")

    
    tests_df = pd.read_csv(TESTS_PERFORMED)
    tests_df.to_csv(f"{S3_FILE_PATH}county-tests-performed-rshiny.csv")

    persons_df = pd.read_csv(PERSONS_TESTED)
    persons_df.to_csv(f"{S3_FILE_PATH}county-persons-tested-rshiny.csv")
    

download_from_rshiny()

## Append the testing by neighborhood

In [2]:
NEIGHBORHOOD_APPENDED = f"{S3_FILE_PATH}la-county-neighborhood-testing-appended.parquet"

# Append dfs
df = pd.read_parquet(NEIGHBORHOOD_APPENDED)

today = datetime.today()
month = today.strftime('%m')
day = today.strftime("%d")

# Just append the single day to the appended df
day_df = pd.read_parquet(f"{S3_FILE_PATH}la-county-neighborhood-testing-2020-{month}-{day}.parquet")
df = df.append(day_df).drop_duplicates()
    
df.to_parquet(NEIGHBORHOOD_APPENDED)

# At this point, this neighborhood URL can be read in neighborhood_utils
# The NEIGHBORHOOD_APPENDED file gets cleaned, cols dropped
# Run notebooks/neighborhood-charts after this, because the neighborhood_utils will write to parquet